In [ ]:
# pip install opencv-python tensorflow keras scikit-learn matplotlib

In [ ]:
# Import essential libraries in the notebook

import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Initialize the webcam using OpenCV

cap = cv2.VideoCapture(0)

In [ ]:
# Capture frames

ret, frame = cap.read()

In [ ]:
# code for capturing and saving images

import cv2
import os

# Create directories for each person (label)
label = "Arya"  # Change this for different classes
output_dir = f"./data/{label}"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(0)
print("Press 's' to save the image, 'q' to quit.")

image_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Webcam", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):  # Save the frame
        image_path = os.path.join(output_dir, f"{label}_{image_count}.jpg")
        cv2.imwrite(image_path, frame)
        image_count += 1
        print(f"Saved: {image_path}")
    
    elif key == ord('q'):  # Quit the capture
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Path to the directory where images are saved
image_dir = "./data/Arya"

In [ ]:
# Get the latest saved image from the directory
images = sorted(os.listdir(image_dir), key=lambda x: os.path.getctime(os.path.join(image_dir, x)))
if len(images) == 0:
    print("No images found in the directory.")
    exit()

latest_image_path = os.path.join(image_dir, images[-1])
print(f"Using image: {latest_image_path}")

In [ ]:
# Read the latest image
image = cv2.imread(latest_image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Load the Haar Cascade face detection model using OpenCV.
# This model will be used to detect faces in the captured images.

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Perform face detection
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [ ]:
# Perform face detection with adjusted parameters
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.05,  # Reduce the scale step for finer detection
    minNeighbors=6,    # Increase for stricter face detection
    minSize=(50, 50)   # Minimum face size to detect
)

In [ ]:
# Draw bounding boxes around detected faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

In [ ]:
# Display the image with bounding boxes
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

In [ ]:
# Draw bounding boxes and add labels
if len(faces) == 0:
    print("No faces detected.")
else:
    for (x, y, w, h) in faces:
        # Draw rectangle with custom color and thickness
        color = (0, 255, 0)  # Green color for the box
        thickness = 2
        cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
        
        # Add label above the rectangle
        label = "Face"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        label_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
        
        label_x = x
        label_y = y - 10 if y - 10 > 10 else y + 10  # Adjust position if label goes out of bounds
        cv2.rectangle(image, (label_x, label_y - label_size[1] - 2), (label_x + label_size[0], label_y + 2), color, cv2.FILLED)
        cv2.putText(image, label, (label_x, label_y), font, font_scale, (0, 0, 0), font_thickness)

# Display the image with bounding boxes and labels
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

In [ ]:
output_dir = './processed_faces/'  # Directory to save preprocessed faces
os.makedirs(output_dir, exist_ok=True)

if len(faces) == 0:
    print("No faces detected.")
else:
    # Loop through detected faces
    for idx, (x, y, w, h) in enumerate(faces):
        # Crop the face
        cropped_face = image[y:y+h, x:x+w]
        
        # Resize to a uniform size (e.g., 160x160 for models like FaceNet)
        resized_face = cv2.resize(cropped_face, (160, 160))
        
        # Save the preprocessed face
        face_path = os.path.join(output_dir, f"face_{idx}.jpg")
        cv2.imwrite(face_path, resized_face)
        print(f"Saved preprocessed face: {face_path}")
        
        # Display the preprocessed face (optional)
        cv2.imshow("Preprocessed Face", resized_face)
        cv2.waitKey(0) # The 0 argument tells OpenCV to wait indefinitely for the user to press any key

cv2.destroyAllWindows()

In [ ]:
# Load a pre-trained model

from tensorflow.keras.applications import MobileNetV2

model = MobileNetV2(weights='imagenet', 
                    include_top=False, 
                    input_shape=(160, 160, 3))

# If you have downloaded a model
# model = load_model('model.h5')  

In [ ]:
# Initialize lists for embeddings and labels
embeddings = []
labels = []

In [ ]:
# define the folder for preprocessed faces
processed_faces_dir = './processed_faces'

# Generate embeddings for all preprocessed faces

# Iterate through all images in the directory
for image_name in os.listdir(processed_faces_dir):
    image_path = os.path.join(processed_faces_dir, image_name)

    # Check if the file is a valid image (optional: you can adjust based on file types)
    if image_name.endswith('.jpg') or image_name.endswith('.jpeg') or image_name.endswith('.png'):
        # Load and preprocess image
        face_image = cv2.imread(image_path)
        face_image = cv2.resize(face_image, (160, 160))  # Ensure correct input size
        face_preprocessed = face_image.astype('float32') / 255.0  # Normalize
        
        # Generate embedding
        embedding = model.predict(np.expand_dims(face_preprocessed, axis=0))[0]
        embeddings.append(embedding)
        
        # Use the image filename as the label (without file extension)
        label = os.path.splitext(image_name)[0]
        labels.append(label)

print(f"Generated embeddings for {len(embeddings)} faces.")

In [ ]:
labels

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(embeddings, labels)
print("Classifier trained.")

In [ ]:
# Test on a new face
new_face = cv2.imread('./processed_faces/test_face.jpg')
new_embedding = get_embedding(facenet_model, new_face)

# Predict the identity
predicted_label = knn.predict([new_embedding])
print("Predicted Identity:", predicted_label[0])